### Watermark of File

In [1]:
# https://github.com/rasbt/watermark#installation-and-updating
# need to pip install watermark
import watermark 

In [2]:
%load_ext watermark

In [3]:
%watermark

Last updated: 2022-04-17T10:19:20.704692-07:00

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 8.2.0

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 21.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 12
Architecture: 64bit



In [4]:
%watermark --iversions

watermark: 2.3.0



### Printing Tables

Let's say we have the following dictionary we want to print as a table. 

In [5]:
d = {0:['This','example'],
     1:['shows','how'],
     2:['to','print'],
     3:['a','table']}

At a high level, the basic idea is to print a f-string representing the header, print a horizontal dividing line, and then print a f-string for each row in succesion by setting up the column features. For this all to work, though, you need to determine an amount of horizontal space each column will consume. I do this in five steps:

1. Determine the number of columns.
2. Determine the amount of blank space between each column, and if you want to use column separators.
3. Determine the header row, which allows you to obtain the length of each header.
4. Determine the maximum length of table entries in each row.
5. Determine the maximum horizontal space contained in each column. 

For Step (1) in this example, I know I want to print the key and each entry of the resulting list as separate columns; this means I we will have three columns (1 for the key, and 2 for the length of the lists). For Step (2), I know I want 5 spaces between each column and no column separators. For the headers in Step (3), I will make them 'Key', 'Column 1', and 'Column 2', respectively, which gives me header lengths of 3, 8, and 8. In Step (4), considering all the lists, the maximum word length in position 0 is 5 and the maximum word length in position 1 is 7, and the length of each key is 1.

This brings us to Step (5). We need to think of the table in terms of characters in each line of the print. Consider the layout below that documents the length of each row. The + signs document blank space in the final table. 

```
Key+++++Column 1+++++Column 2
-----------------------------
0+++++++This+++++++++example+
1+++++++shows++++++++how+++++
2+++++++to+++++++++++print+++
3+++++++a++++++++++++table+++
```

In my experience, it is easiest to think about the header row in chunks; the number of chunks is equal to the number of columns, and length of each chunk is the length of the header plus the space between columns. So in our example, you get three chunks with lengths as shown:

```
Key+++++
len(Key+++++) = 8

Column 1+++++
len(Column 1+++++) = 13

Column 2
len(Column 2) = 8
```

Another way I think about this is taking the maximum length in each column and adding the amount of white space between each column to each of these values. For the ```Key``` column we have ```max(1, 3) + 5 = 8```, for ```Column 1``` we have ```max(8, 5) + 5 = 13```, and for ```Column 2``` we have ```max(8, 7) + 5 = 13```. The one slight adjustment you might make is on the 
last column; there really isn't a reason to extend the space in the last column if you won't use it, so I would adjust the calculation on ```Column 2``` to ```max(8, 7) = 8```.

With all of the information, now we can format the f-strings. For the purpose of printing a table, the key is using a collection of structures that look like ```{str_var:<#}```. This structure tells the f-string to left justify (```<```) a block of characters of length ```#``` to display ```str_var```, which could be a variable or an actual string. When we use this structure, if ```len(str_var) < #```, then blank space equal to ```# - len(str_var)``` will be inserted to give a total length of ```#```. The line of dashes below the header should be the total length of all columns.

For the headers row, each piece is ```{header_label:<horizontal_space}```. The length of the dashed line is the cumulative horizontal space, and each row of the table is formatted like the header row. This gives us the following code.

In [6]:
d = {0:['This','example'],
     1:['shows','how'],
     2:['to','print'],
     3:['a','table']}

print(f'{"Key":<8}{"Column 1":<13}{"Column 2":<8}')
print('-' * (8 + 13 + 8))
for k, v in d.items():
    print(f'{k:<8}{v[0]:<13}{v[1]:<8}')

Key     Column 1     Column 2
-----------------------------
0       This         example 
1       shows        how     
2       to           print   
3       a            table   


You could adjust this example to include column separators and fit the columns. One such implementation would be the following. You could also center entries (changing ```<``` to ```^```) in the 'cells'.

In [7]:
d = {0:['This','example'],
     1:['shows','how'],
     2:['to','print'],
     3:['a','table']}

print('-' * (5 + 1 + 10 + 1 + 10 + 2))
print(f'{"|":1}{" Key ":^5}{"|":1}{" Column 1 ":<10}{"|":1}{" Column 2 ":<10}{"|":1}')
print('-' * (5 + 1 + 10 + 1 + 10 + 2))
for k, v in d.items():
    print(f'{"|":1}{k:^5}{"|":1}{v[0]:^10}{"|":1}{v[1]:^9} {"|":1}')
print('-' * (5 + 1 + 10 + 1 + 10 + 2))

-----------------------------
| Key | Column 1 | Column 2 |
-----------------------------
|  0  |   This   | example  |
|  1  |  shows   |   how    |
|  2  |    to    |  print   |
|  3  |    a     |  table   |
-----------------------------


Hopefully you can see why I opt for no separators or borders, but choose to slightly increase the space between each column. These features add hard-to-read formatting in the f-string and make the table hard to read once printed.

In this example, I knew the structure of the dictionary because it was so small, but if you didn't, you could write a function to loop over the elements or other variables you have and get the lengths of each element for printing. If you had to do this often, it would be a better solution, but for a one-off, what I did above will suffice.